## 0. Getting setup

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt

from torch import nn
from torchvision import transforms

print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

try:
    from torchinfo import summary
except:
    print("[INFO] Coudnl't find torchinfo... installing it")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from going_modular import data_setup, engine
except:
    print("[INFO] Couldn't find going_modular scripts...")


torch version: 2.2.0+cu121
torchvision version: 0.17.0+cu121
[INFO] Coudnl't find torchinfo... installing it


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

# 1. Get data

In [16]:
import os
import zipfile

from pathlib import Path

import requests

def download_data(source: str, 
                  destination: str,
                  remove_source: bool = True) -> Path:
    # Setup path to data folder
    data_path = Path("data/")
    image_path = data_path / destination

    # If the image folder doesn't exist, download it and prepare it... 
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory exists, skipping download.")
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)
        
        # Download pizza, steak, sushi data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)

        # Unzip pizza, steak, sushi data
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...") 
            zip_ref.extractall(image_path)

        # Remove .zip file
        if remove_source:
            os.remove(data_path / target_file)
    
    return image_path

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] Did not find data\pizza_steak_sushi directory, creating one...
[INFO] Downloading pizza_steak_sushi.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip...
[INFO] Unzipping pizza_steak_sushi.zip data...


WindowsPath('data/pizza_steak_sushi')

# 2. Create Datasets and DataLoaders

In [19]:
train_dir = image_path / "train"
test_dir = image_path / "test"

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.226])

manual_transforms = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize])
print(f"Manually created transforms: {manual_transforms}")

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=manual_transforms,
    batch_size=32
)

train_dataloader, test_dataloader, class_names

Manually created transforms: Compose(
    Resize(size=(224, 224), interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.226])
)


(<torch.utils.data.dataloader.DataLoader at 0x1d515fe4590>,
 ['pizza', 'steak', 'sushi'])

In [21]:
model = torchvision.models.efficientnet_b0(pretrained=True).to(device) # for torchvision < 13
# weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT for torchvision > 0.13
# model = torchvision.models.efficientnet_b0(weights=weights)

C:\Users\Kajetan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Kajetan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\Kajetan/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:02<00:00, 10.2MB/s]


In [22]:
# Freeze all base layers
for param in model.features.parameters():
    param.requires_grad = False

set_seeds()

model.classfier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280,
              out_features=len(class_names),
              bias=True).to(device))

In [25]:
# from torchinfo import summary

# summary(model,
#         input_size=(32, 3, 224, 224),
#         verbose=0,
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
#        )

# 4. Train model and track results

In [47]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [49]:
try:
    import wandb
except:
    !pip install wandb
    import wandb

In [60]:
from typing import Dict, List
from tqdm.auto import tqdm
from going_modular.engine import train_step, test_step

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          criterion: torch.nn.Module,
          epochs: int,
          device: torch.device,
         wandb_project_name: str = None,
         wandb_run_name: str = None) -> Dict[str, List]:
    
    if wandb_project_name is not None:
        wandb.init(project=wandb_project_name, name=wandb_project_name)
    
    results = {"train_loss": [],
               "test_loss": [],
               "train_acc": [],
               "test_acc": []
              }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=test_dataloader,
                                           loss_fn=criterion,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                         dataloader=test_dataloader,
                                         loss_fn=criterion,
                                         device=device)
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_acc: {test_acc:.4f} | "
        )

        if wandb_project_name is not None:
            wandb.log({"train_loss": train_loss,
            "test_loss": test_loss,
            "train_acc": train_acc,
            "test_acc": test_acc})
            
        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)
        results["train_acc"].append(train_acc)
        results["test_acc"].append(test_acc)

    return results

In [68]:
set_seeds()
model0 = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                criterion=criterion,
                epochs=20,
                device=device,
               wandb_project_name="torch_experiment_track",
               wandb_run_name="torch_experiment_v1")

test_acc,▁▃▅▅▆▇████
test_loss,█▆▅▄▄▃▂▂▁▁
train_acc,▁▅▅▆▆▆▆▆▇█
train_loss,█▆▅▄▃▃▂▂▁▁
test_acc,0.63731
test_loss,1.47138
train_acc,0.64678
train_loss,1.33514


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.2614 | test_loss: 1.4073 | train_acc: 0.5350 | test_acc: 0.6686 | 
Epoch: 2 | train_loss: 1.1727 | test_loss: 1.3568 | train_acc: 0.6989 | test_acc: 0.6790 | 
Epoch: 3 | train_loss: 1.2345 | test_loss: 1.3337 | train_acc: 0.6572 | test_acc: 0.6790 | 
Epoch: 4 | train_loss: 1.2629 | test_loss: 1.2984 | train_acc: 0.6373 | test_acc: 0.6894 | 
Epoch: 5 | train_loss: 1.1265 | test_loss: 1.2734 | train_acc: 0.6070 | test_acc: 0.6998 | 
Epoch: 6 | train_loss: 1.1345 | test_loss: 1.2212 | train_acc: 0.6364 | test_acc: 0.6799 | 
Epoch: 7 | train_loss: 1.0738 | test_loss: 1.1959 | train_acc: 0.7311 | test_acc: 0.6799 | 
Epoch: 8 | train_loss: 1.0301 | test_loss: 1.1768 | train_acc: 0.6278 | test_acc: 0.7008 | 
Epoch: 9 | train_loss: 0.9603 | test_loss: 1.1387 | train_acc: 0.6799 | test_acc: 0.7008 | 
Epoch: 10 | train_loss: 0.9565 | test_loss: 1.1047 | train_acc: 0.7311 | test_acc: 0.7112 | 
Epoch: 11 | train_loss: 0.9266 | test_loss: 1.0724 | train_acc: 0.6477 | test_a

In [70]:
data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")

data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

[INFO] data\pizza_steak_sushi directory exists, skipping download.
[INFO] Did not find data\pizza_steak_sushi_20_percent directory, creating one...
[INFO] Downloading pizza_steak_sushi_20_percent.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip...
[INFO] Unzipping pizza_steak_sushi_20_percent.zip data...


In [72]:
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

test_dir = data_10_percent_path / "test"

print(f"Training_directory 10%: {train_dir_10_percent}")
print(f"Training_directory 20%: {train_dir_20_percent}")
print(f"Testing_directory {test_dir}")

Training_directory 10%: data\pizza_steak_sushi\train
Training_directory 20%: data\pizza_steak_sushi_20_percent\train
Testing_directory data\pizza_steak_sushi\test


In [74]:
from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

In [86]:
BATCH_SIZE = 32

train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir_10_percent,
    test_dir=test_dir,
    transform=simple_transform,
    batch_size=BATCH_SIZE
)
train_dataloader_20_percent, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir_20_percent,
    test_dir=test_dir,
    transform=simple_transform,
    batch_size=BATCH_SIZE
)
print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(test_dataloader)} (all experiments will use the same test data)")
print(f"Number of classes {len(class_names)}, class_names {class_names}")

Number of batches of size 32 in 10 percent training data: 8
Number of batches of size 32 in 20 percent training data: 15
Number of batches of size 32 in testing data: 3 (all experiments will use the same test data)
Number of classes 3, class_names ['pizza', 'steak', 'sushi']


In [94]:
import torchvision
from torchinfo import summary

effnetb2 = torchvision.models.efficientnet_b2(pretrained=True).to(device)

# summary(model=effnetb2,
#         input_size=(32, 3, 224, 224),
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
# )

print(f"Number of in_features to final layer of EfficientNetB2; {len(effnetb2.classifier.state_dict()['1.weight'][0])}")

Number of in_features to final layer of EfficientNetB2; 1408


In [122]:
import torchvision
from torch import nn

OUT_FEATURES = len(class_names)

def create_effnetb0():
    model = torchvision.models.efficientnet_b0(pretrained=True).to(device)

    for param in model.features.parameters():
        param.requires_grad = False

    set_seeds()

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(in_features=1280, out_features=OUT_FEATURES)
    ).to(device)

    model.name = "effnetb0"
    print(f"[INFO] Created new {model.name} model.")
    return model

def create_effnetb2():
    model = torchvision.models.efficientnet_b2(pretrained=True).to(device)

    for param in model.features.parameters():
        param.requires_grad = False

    set_seeds()

    model.classfier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1408, out_features=OUT_FEATURES)
    ).to(device)

    model.name = "effnetb2"
    print(f"[INFO] Created new {model.name} model.")
    return model

In [124]:
effnetb0 = create_effnetb0()

# summary(model=effnetb0,
#         input_size=(32, 3, 224, 224),
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
#        )

[INFO] Created new effnetb0 model.


In [128]:
effnetb2 = create_effnetb2()

# summary(model=effnetb2,
#         input_size=(32, 3, 224, 224),
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
#        )

[INFO] Created new effnetb2 model.


In [134]:
num_epochs = [5, 10]
models = ["effnetb0", "effnetb2"]
train_dataloaders = {"data_10_percent": train_dataloader_10_percent,
                     "data_20_percent": train_dataloader_20_percent}

In [144]:
# %%time
# from going_modular.utils import save_model

# set_seeds(seed=42)

# experiment_number = 0

# for dataloader_name, train_dataloader in train_dataloaders.items():
#     for epochs in num_epochs:
#         for model_name in models:
#             experiment_number += 1
#             print(f"[INFO] Experiment number: {experiment_number}")
#             print(f"[INFO] Model: {model_name}")
#             print(f"[INFO] DataLoader: {dataloader_name}")
#             print(f"[INFO] Number of epochs: {epochs}")

#             if model_name == "effnetb0":
#                 model = create_effnetb0()
#             else:
#                 model = create_effnetb2()

#             criterion = nn.CrossEntropyLoss()
#             optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

#             train(model=model,
#                 train_dataloader=test_dataloader,
#                 test_dataloader=test_dataloader,
#                 optimizer=optimizer,
#                 criterion=criterion,
#                 epochs=epochs,
#                 device=device,
#                 wandb_project_name=model_name,
#                 wandb_run_name=model_name)

#             save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
#             save_model(model=model,
#                         target_dir="models",
#                         model_name=save_filepath)
#             print("-"*50+"\n")

In [152]:
train(model=effnetb0,
      train_dataloader=train_dataloader,
      test_dataloader=test_dataloader,
      optimizer=torch.optim.Adam(params=effnetb0.parameters(), lr=1e-2),
      criterion=criterion,
      epochs=20,
      device=device,
      wandb_project_name="effnetb0v1")

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 2.4582 | test_loss: 0.9101 | train_acc: 0.0521 | test_acc: 0.5388 | 
Epoch: 2 | train_loss: 0.4887 | test_loss: 0.8324 | train_acc: 0.8854 | test_acc: 0.6146 | 
Epoch: 3 | train_loss: 1.1438 | test_loss: 0.6522 | train_acc: 0.6250 | test_acc: 0.7083 | 
Epoch: 4 | train_loss: 1.0308 | test_loss: 0.4101 | train_acc: 0.6278 | test_acc: 0.8854 | 
Epoch: 5 | train_loss: 0.7541 | test_loss: 0.3495 | train_acc: 0.6600 | test_acc: 0.9489 | 
Epoch: 6 | train_loss: 0.5150 | test_loss: 0.2837 | train_acc: 0.7746 | test_acc: 0.9176 | 
Epoch: 7 | train_loss: 0.3674 | test_loss: 0.2477 | train_acc: 0.9167 | test_acc: 0.9375 | 
Epoch: 8 | train_loss: 0.3842 | test_loss: 0.2143 | train_acc: 0.8333 | test_acc: 0.9479 | 
Epoch: 9 | train_loss: 0.4335 | test_loss: 0.1539 | train_acc: 0.8021 | test_acc: 0.9896 | 
Epoch: 10 | train_loss: 0.3646 | test_loss: 0.1408 | train_acc: 0.8438 | test_acc: 0.9896 | 
Epoch: 11 | train_loss: 0.3158 | test_loss: 0.1405 | train_acc: 0.9479 | test_a

{'train_loss': [2.45819620291392,
  0.4886525273323059,
  1.1438140620787938,
  1.0307846864064534,
  0.7541138132413229,
  0.514960765838623,
  0.36741480231285095,
  0.3841744164625804,
  0.43349965910116833,
  0.3646253744761149,
  0.3157743116219838,
  0.26548127830028534,
  0.24175597230593363,
  0.22375581910212836,
  0.22200021396080652,
  0.22368882099787393,
  0.1908637434244156,
  0.19146337111790976,
  0.17925211787223816,
  0.18196836113929749],
 'test_loss': [0.9101417859395345,
  0.8324042558670044,
  0.6521581063667933,
  0.4101025735338529,
  0.3495146880547206,
  0.28372859954833984,
  0.24766658246517181,
  0.21431847661733627,
  0.1539208466808001,
  0.14081821093956629,
  0.1405228798588117,
  0.11567142729957898,
  0.09942262743910153,
  0.09087116892139117,
  0.08355570336182912,
  0.07634762426217397,
  0.07204516232013702,
  0.06607562613983949,
  0.06140480128427347,
  0.05734508919219176],
 'train_acc': [0.052083333333333336,
  0.8854166666666666,
  0.625,
  0

In [170]:
def save_model(model, model_name):
    file_path = f"models/{model_name}.pth"

    torch.save({
        'model_state_dict': model.state_dict(),
        'model_name': model_name
    }, file_path)

    print(f"[INFO] Model '{model_name}' saced successfully to '{file_path}'")

save_model(effnetb0, "effnetb0V1")

[INFO] Model 'effnetb0V1' saced successfully to 'models/effnetb0V1.pth'


In [219]:
def load_model(filepath):
    checkpoint = torch.load(filepath)

    if checkpoint['model_name'] == "effnetb0V1":
        model = create_effnetb0()
    elif checkpoint['model_name'] == "effnetb0V2":
        model = create_effnetb2()
    else:
        raise ValueError("Unsupported model name")

    model.load_state_dict(checkpoint['model_state_dict'])

    print(f"[INFO] Model '{checkpoint['model_name']}' loaded successfully")
    return model

effnetV1_model_path = "models/effnetb0V1.pth"
effnetV1_loaded = load_model(effnetV1_model_path)

[INFO] Created new effnetb0 model.
[INFO] Model 'effnetb0V1' loaded successfully


In [221]:
# Check the model file size
from pathlib import Path

effnetb0_model_size = Path(effnetV1_model_path).stat().st_size // (1024*1024)
print(f"Effnetb0V1_loaded feature extractor model size: {effnetb0_model_size} MB")

Effnetb0V1_loaded feature extractor model size: 15 MB


In [254]:
from going_modular.predictions import pred_and_plot_image
import random
from pathlib import Path

# Define the image size
image_size = (224, 224)

# Get all test image paths from the 20% dataset
test_image_path_list = list(Path(data_20_percent_path / "test").glob("*/*.jpg"))

# Randomly select k number of images
num_images_to_plot = 3
test_image_path_sample = random.sample(test_image_path_list, k=num_images_to_plot)

# Iterate through random test image paths, make predictions on them, and plot them
for image_path in test_image_path_sample:
    pred_and_plot_image(model=effnetV1_loaded,
                        image_path=image_path,
                        class_names=class_names,
                        image_size=image_size)


TypeError: Size should be int or sequence. Got <class 'pathlib.WindowsPath'>